In [43]:
import os
import base64
import requests
import json
from mistralai import Mistral

# API ключ для Vision LLM
api_key = "BsoGgWX10TneaJMu89JevSiGCjU3ODnG"

# Функция для кодирования изображения в base64
def encode_image(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Функция для создания JSON промпта для анализа текста
def create_json_prompt_for_text_analysis(text):
    prompt = {
        "text": text,
        "entities": [
            {"entity": "address", "types": ["address"]},
            {"entity": "animal type", "types": ["animal type"]},
            {"entity": "status", "types": ["status"]},
            {"entity": "description", "types": ["description"]}
        ]
    }
    return json.dumps(prompt)

# Функция для взаимодействия с Vision LLM
def get_vision_llm_response(text):
    model = "pixtral-12b-2409"
    client = Mistral(api_key=api_key)

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""
**Objective:**
Analyze the given text and extract the address, animal type, status, and description.

**Format Requirements:**
- Do not explain your answer. Just output the JSON.

**Entity Annotation Details:**
- The entity "animal type" must be either "Кошка" or "Собака".
- The entity "status" must be either "Найдена" or "Пропала".

**Example:**

text = "Пропала собака, улица Ленина, дом 5, Санкт-Петербург. Очень дружелюбная собака, черного цвета, пропала во дворе, возможно, выбежала на улицу."


```json
{{
    "Адрес": "Санкт-Петербург, улица Ленина, дом 5",
    "Тип живтоного": "Собака",
    "Статус": "Пропала",
    "Описание": "Очень дружелюбная собака, черного цвета, пропала во дворе, возможно, выбежала на улицу."
}}
```

Text:
"{text}"
"""
                }
            ]
        }
    ]

    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )

    return chat_response.choices[0].message.content

# Пример использования функции
text = "Пропала кошка, Пенягинская ул. 16, Москва. Очень добрая кошка, домашняя, в один момент пропала из поля зрения и так и не объявилась , могла выбежать в дверь или вылезти из окна, за находку полагается награда, очень ждём свою кошечку дома."

# Получение ответа от Vision LLM
llm_response = get_vision_llm_response(text)
print(f"LLM Response: {llm_response}")

# Парсинг JSON ответа
try:
    # Удаляем маркеры для форматирования в Markdown
    llm_response = llm_response.replace("```json", "").replace("```", "")
    llm_data = json.loads(llm_response)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
    llm_data = {}

# Вывод извлеченных данных
address = None
animal_type = None
status = None
description = None

address = llm_data.get('Адрес')
animal_type = llm_data.get('Тип животного') if llm_data.get('Тип животного') in ['Кошка', 'Собака'] else None
status = llm_data.get('Статус') if llm_data.get('Статус') in ['Найдена', 'Пропала'] else None
description = llm_data.get('Описание')

print(f"Address: {address}")
print(f"Animal Type: {animal_type}")
print(f"Status: {status}")
print(f"Description: {description}")


LLM Response: ```json
{
    "Адрес": "Москва, Пенягинская ул. 16",
    "Тип животного": "Кошка",
    "Статус": "Пропала",
    "Описание": "Очень добрая кошка, домашняя, в один момент пропала из поля зрения и так и не объявилась, могла выбежать в дверь или вылезти из окна, за находку полагается награда, очень ждём свою кошечку дома."
}
```
Address: Москва, Пенягинская ул. 16
Animal Type: Кошка
Status: Пропала
Description: Очень добрая кошка, домашняя, в один момент пропала из поля зрения и так и не объявилась, могла выбежать в дверь или вылезти из окна, за находку полагается награда, очень ждём свою кошечку дома.


In [34]:
import pandas as pd

# Загрузка данных из CSV файла
links_df = pd.read_csv('links.csv')
links_df['imgs'] = links_df['imgs'].apply(eval).tolist()  # Преобразуем все строки в списки


# Фильтрация данных на основе типа животного и статуса
if animal_type == 'Кошка':
    if status == 'Пропала':
        filtered_data = links_df[(links_df['status'] == 'Найдена') & (links_df['animal_type'] == 'Кошка')]
    elif status == 'Найдена':
        filtered_data = links_df[(links_df['status'] == 'Пропала') & (links_df['animal_type'] == 'Кошка')]
elif animal_type == 'Собака':
    if status == 'Пропала':
        filtered_data = links_df[(links_df['status'] == 'Найдена') & (links_df['animal_type'] == 'Собака')]
    elif status == 'Найдена':
        filtered_data = links_df[(links_df['status'] == 'Пропала') & (links_df['animal_type'] == 'Собака')]
else:
    filtered_data = pd.DataFrame()  # Пустой DataFrame, если условия не выполнены

# Вывод отфильтрованных данных
print("Отфильтрованные данные:")
filtered_data.head()


Отфильтрованные данные:


,link,imgs,title,description,latitude,longitude,address,animal_type,status
7,https://pet911.ru/moskva/found/cat/rf925772,[https://cdn.pet911.ru/1731147488672f36e0eb1c4...,"Кот найден: 3-я Рыбинская ул, 1, Москва",Подбросили кота,55.791159,37.660426,"3-я Рыбинская улица, 1, Москва",Кошка,Найдена
9,https://pet911.ru/moskovskaya-oblast/found/cat...,[https://cdn.pet911.ru/1731145021672f2d3d84b99...,"Кошка найдена: Малые Вязёмы, МО","Серый кот, мальчик, не кастрирован, регулярно ...",55.628642,37.027815,"Малые Вязёмы, Московская область",Кошка,Найдена
10,https://pet911.ru/dubna/found/cat/rf925734,[None],"Кошка найдена в СНТ Репка, Дубна, 30",Британец мальчик об стриженный найденСНТ Репка...,55.122666,37.346480,"30, Дубна",Кошка,Найдена
25,https://pet911.ru/moskva/found/cat/rf925423,[https://cdn.pet911.ru/1731060784672de4306a77f...,"Кошка найдена: Дербышевский 24, Томск","НАЙДЕН ПИТОМЕЦ ТОМСК ДЕРБЫШЕВСКИЙ 24, ОСТАВЛЕН...",55.753593,37.621489,"Красная площадь, Москва",Кошка,Найдена
33,https://pet911.ru/moskva/found/cat/rf925348,[https://cdn.pet911.ru/1731044273672da3b131702...,"Кошка найдена: Севастопольский пр-т, 7 к2",На вид девочка. Возраст около трех лет,55.691261,37.608949,"Севастопольский проспект, 7 к2, Москва",Кошка,Найдена


In [35]:
import requests

# Функция для геокодирования адреса с использованием Nominatim API
def geocode_address(address):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'PetFinderApp/1.0 (anton.epub@gmail.com)'
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Проверяем, что запрос успешен
        data = response.json()

        if data:
            location = data[0]
            latitude = float(location['lat'])
            longitude = float(location['lon'])
            return latitude, longitude
        else:
            print("Адрес не найден")
            return None
    except requests.exceptions.HTTPError as e:
        print(f"Ошибка HTTP: {e}")
        print(f"Текст ответа: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        return None
    except ValueError as e:
        print(f"Ошибка парсинга JSON: {e}")
        print(f"Текст ответа: {response.text}")
        return None

# Геокодируем адрес и получаем GPS координаты
if address:
    coordinates = geocode_address(address)
    if coordinates:
        latitude, longitude = coordinates
        print(f"Широта: {latitude}, Долгота: {longitude}")

        # Сортируем отфильтрованные данные по удаленности от полученных координат
        filtered_data['distance'] = ((filtered_data['latitude'] - latitude) ** 2 + (filtered_data['longitude'] - longitude) ** 2) ** 0.5
        sorted_data = filtered_data.sort_values(by='distance')

        # Выводим отсортированные данные
        print("Отсортированные данные по удаленности:")
        print(sorted_data.head())
    else:
        print("Не удалось получить координаты для данного адреса.")
else:
    print("Адрес не указан для геокодирования.")


Широта: 55.83544695, Долгота: 37.36016179987142
Отсортированные данные по удаленности:
                                             link  \
3104  https://pet911.ru/moskva/found/cat/rf885971   
2768  https://pet911.ru/moskva/found/cat/rf890666   
103   https://pet911.ru/moskva/found/cat/rf924044   
753   https://pet911.ru/moskva/found/cat/rf915866   
310   https://pet911.ru/moskva/found/cat/rf920810   

                                                   imgs  \
3104  [https://cdn.pet911.ru/172396384366c199c34ba97...   
2768  [https://cdn.pet911.ru/172466639366cc52196b458...   
103   [https://cdn.pet911.ru/1730705648672878f0a0053...   
753   [https://cdn.pet911.ru/1729082007670fb2976c92a...   
310   [https://cdn.pet911.ru/1730038918671e4c864f1c5...   

                                             title  \
3104  Найдена кошка, ул. Генерала Белобородова, 20   
2768     Найдена кошка на ул. Рословка, 10, Москва   
103     Найдена русская голубая кошка, Барышиха 16   
753      Кошка найдена 

C:\Users\admin\AppData\Local\Temp\ipykernel_15848\1323174960.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['distance'] = ((filtered_data['latitude'] - latitude) ** 2 + (filtered_data['longitude'] - longitude) ** 2) ** 0.5


In [42]:
import os
from mistralai import Mistral

# Retrieve the API key from environment variables
api_key = "BsoGgWX10TneaJMu89JevSiGCjU3ODnG"

# Specify model
model = "pixtral-12b-2409"

# Initialize the Mistral client
client = Mistral(api_key=api_key)

def compare_animal(record, target_description):
    print(f"Сравниваем животное с описанием: {target_description}")
    print(f"Используем изображения: {record['imgs']}")

    # Подготавливаем сообщения для LLM
    messages = [
        {
            "role": "user",
            "content": []
        }
    ]
    
    # Добавляем текстовое сообщение
    messages[0]['content'].append({
        "type": "text",
        "text": f"Определите, найденное животное, это искомое или другое. Описание искомого животного: {target_description}"
    })

    # Добавляем текстовое сообщение
    messages[0]['content'].append({
        "type": "text",
        "text": f"Описание найденного живтного: {record['description']}"
    })

    # Итерируемся по фотографиям и добавляем сообщения
    for img in record['imgs']:
        print(f"Изображение: {img}")
        messages[0]['content'].append({
            "type": "image_url",
            "image_url": img
        })

    # Добавляем текстовое сообщение
    messages[0]['content'].append({
        "type": "text",
        "text": f'После рассуждений, напишите ответ в формате: Ответ: {{"result": "искомое животное" или "другое животное"}}'
    })


    # Получаем ответ от LLM
    print("Отправляем запрос к LLM...")
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    print("Получен ответ от LLM.")

    # Возвращаем ответ в формате JSON
    return chat_response.choices[0].message.content

# Цикл для обработки первых 7 записей
results = []
for index, record in sorted_data.head(7).iterrows():
    print(f"Обрабатываем запись {index + 1}: {record['title']}")
    target_description = record['description']  # Используем описание из текущей записи
    result = compare_animal(record, target_description)
    
    # Изменяем способ извлечения JSON строки с использованием split
    json_string = result.split("Ответ:")[-1].strip()  # Извлекаем часть строки после слова "Ответ:" и убираем лишние пробелы
    print(f"JSON строка: {json_string}")
    json_result = json.loads(json_string)  # Преобразуем строку в JSON
    results.append(json_result['result'])  # Берем только вывод
    print(f"Результат для записи {index + 1}: {json_result['result']}")

# Выводим результаты
print("Все результаты:")
print(results)


Обрабатываем запись 3105: Найдена кошка, ул. Генерала Белобородова, 20
Сравниваем животное с описанием: Нашли ночью с 17 на 18, выглядит как домашняя, как будто совсем недавно ушла из дома. Окрас черепашковый с белыми пузом и ножками
Используем изображения: ['https://cdn.pet911.ru/172396384366c199c34ba977.23820572_IMG_20240817_231348.webp', 'https://cdn.pet911.ru/172396383866c199be91ce68.95683467_IMG_20240818_011447.webp', 'https://cdn.pet911.ru/172396384366c199c34ba977.23820572_IMG_20240817_231348.webp', 'https://cdn.pet911.ru/172396383866c199be91ce68.95683467_IMG_20240818_011447.webp']
Изображение: https://cdn.pet911.ru/172396384366c199c34ba977.23820572_IMG_20240817_231348.webp
Изображение: https://cdn.pet911.ru/172396383866c199be91ce68.95683467_IMG_20240818_011447.webp
Изображение: https://cdn.pet911.ru/172396384366c199c34ba977.23820572_IMG_20240817_231348.webp
Изображение: https://cdn.pet911.ru/172396383866c199be91ce68.95683467_IMG_20240818_011447.webp
Отправляем запрос к LLM...
По